In [1]:
import pandas as pd
df = pd.read_csv('./data/ccks_2017_train.csv')
df

,Unnamed: 0,text,BIO_anno
0,0,明，女性，26岁，河北省承德市滦平县人，现住河北省承德市双滦区御水花园，主因头部、腹部双下肢...,O O O O O O O O O O O O O O O O O O O O O O O ...
1,1,，中年男性，32岁，生于河北省承德市，现住河北省承德市，主因左肘关节屈曲畸形14个月于201...,O O O O O O O O O O O O O O O O O O O O O O O ...
2,2,，青年男性，25岁，河北省承德市双滦区人，现住河北省承德市双滦区三岔口御景家园，主因外伤后头...,O O O O O O O O O O O O O O O O O O O O O O O ...
3,3,神清语利，自动体位，查体合作。左、右上中切牙缺失，右上侧切牙松动、触痛，下唇右侧内黏膜挫伤，...,O O O O O O O O O O O O O O O B-body I-body I-...
4,4,p；nbsp；右腰部疼痛伴发热12小时于2016--10--213：13收入院,O O O O O O O B-body I-body I-body B-symp I-sy...
...,...,...,...
1555,1555,患者入院后完善相关辅助检查，根据患者出现周身无力，纳差等恶病质表现，查体肝位于右侧季肋下2c...,O O O O O O O O O O O O O O O O O O O O O O O ...
1556,1556,患者入院后完善相关辅助检查，依据患者依据活动后胸闷、气短12年，间断喘憋2年，加重2天入院。...,O O O O O O O O O O O O O O O O O O O O O O O ...
1557,1557,入院后依据病史、症状、体征及辅助检查诊断为：1.颈椎病。2.高血压病。3.缺血性心肌病。入院...,O O O O O O O O O O O O O O O O O O O O O O O ...
1558,1558,依据患者病情、症状、体征及辅助检查诊断为：阵发性室上性心动过速。治疗上给予：1.予以内科II...,O O O O O O O O O O O O O O O O O O O O O O O ...


In [2]:
import synonyms
import jieba

jieba.lcut('直肠癌低位前切除术')

smart_open library not found; falling back to local-filesystem-only


Building prefix dict from F:\anaconda\envs\pytorch\lib\site-packages\synonyms\data\vocab.txt ...
Loading model from cache C:\Users\RL\AppData\Local\Temp\jieba.u3ae10c8cd5d0de0e22f93d188f2f8e5b.cache



 Synonyms: v3.20.0, Project home: https://github.com/chatopera/Synonyms/

 Project Sponsored by Chatopera

  deliver your chatbots with Chatopera Cloud Services --> https://bot.chatopera.com

>> Synonyms load wordseg dict [F:\anaconda\envs\pytorch\lib\site-packages\synonyms\data\vocab.txt] ... 


Loading model cost 1.237 seconds.
Prefix dict has been built successfully.


>> Synonyms on loading stopwords [F:\anaconda\envs\pytorch\lib\site-packages\synonyms\data\stopwords.txt] ...
>> Synonyms on loading vectors [F:\anaconda\envs\pytorch\lib\site-packages\synonyms\data\words.vector.gz] ...


['直肠癌', '低位', '前', '切除术']

In [3]:
synlst = synonyms.nearby('切除术', 2)
synlst

(['切除术', '切除'], [1.0, 0.7716973])

In [7]:
line = df.loc[6]
bio = line['BIO_anno']

start = 0
end = 0
bio_list = bio.split(' ')
for idx in range(len(bio_list)):
    if 'B-' in bio_list[idx]:
        start = idx
    if start != 0 and bio_list[idx] == 'O':
        end = idx
        print(line['text'][start:end])
        print(bio_list[start:end])
        start = end = 0

print(bio)
print(line['text'])

疼痛
['B-symp', 'I-symp']
O O O O B-symp I-symp O O O O O O O O O O O O O O O O O O O O O O O O O O
右侧腰部疼痛10余天于2016--09--0510：45收入院。


In [20]:
type_count = {
    'B-body': 0,
    'B-symp': 0,
    'B-chec': 0,
    'B-dise': 0,
    'B-cure': 0,
}
type_enum = {
    'B-body': [],
    'B-symp': [],
    'B-chec': [],
    'B-dise': [],
    'B-cure': [],
}


for index, row in df.iterrows():
    start = 0
    end = 0
    
    bio = row['BIO_anno']
    bio_list = bio.split(' ')
    for idx in range(len(bio_list)):
        if 'B-' in bio_list[idx]:
            start = idx
        if start != 0 and bio_list[idx] == 'O':
            end = idx
            type_count[bio_list[start]] += 1
            if bio_list[start:end] not in type_enum[bio_list[start]]:
               type_enum[bio_list[start]].append(row['text'][start:end])
            
            start = end = 0

type_count

{'B-body': 9114,
 'B-symp': 8236,
 'B-chec': 11163,
 'B-dise': 1462,
 'B-cure': 3210}